In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
    'lhc_per': 'vid|a3d85efe963899fee2b5',
    '_gid': 'GA1.3.289911229.1685154797',
    '_ga': 'GA1.1.2075372866.1684772919',
    '_ga_3C534XGNC2': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_Q33BVTTDL1': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_QHEB12PC6D': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28F695C90B262BFC00B79BBD6CC1; lhc_per=vid|a3d85efe963899fee2b5; _gid=GA1.3.289911229.1685154797; _ga=GA1.1.2075372866.1684772919; _ga_3C534XGNC2=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_Q33BVTTDL1=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_QHEB12PC6D=GS1.1.1685154797.4.1.1685154882.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies, headers=headers)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [5]:
files = glob('pages-usim/*.json')
len(files)

1178

In [8]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [17]:
filtered_pdfs = []
for p in tqdm(pdfs):
#     if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
#         continue
#     if not p.endswith('.pdf'):
#         continue
#     if 'ade.sagepub.com' in p:
#         continue
    
    if p.startswith('/jspui'):
        p = 'https://oarep.usim.edu.my' + p
        
    if not p.startswith('http'):
        continue
        
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|██████████████████████████████████| 9811/9811 [00:00<00:00, 1755260.05it/s]


9745

In [18]:
filtered_pdfs

['http://adjournal.net/articles/31/315.pdf',
 'http://ajbasweb.com/old/ajbas/2013/June/202-206.pdf',
 'http://aos.usm.my/docs/Vol_16/aos.2020.0057.pdf',
 'http://aos.usm.my/docs/Vol_17_Supp1/AOS.17Supp1.Art02.pdf',
 'http://apjeas.apjmr.com/wp-content/uploads/2018/02/APJEAS-2018.5.1.08.pdf',
 'http://archive.sciendo.com/MJSS/mjss.2018.8.issue-5-1/mjss-2018-0108/mjss-2018-0108.pdf',
 'http://archive.sciendo.com/MJSS/mjss.2018.9.issue-3/mjss-2018-0056/mjss-2018-0056.pdf',
 'http://conference.kuis.edu.my/imam/images/eprosiding/2019/3023-imam-2019.pdf',
 'http://docsdrive.com/pdfs/medwelljournals/jeasci/2018/2077-2080.pdf',
 'http://docsdrive.com/pdfs/medwelljournals/sscience/2015/61-66.pdf',
 'http://docsdrive.com/pdfs/medwelljournals/sscience/2018/1072-1078.pdf',
 'http://docsdrive.com/pdfs/medwelljournals/sscience/2018/1273-1277.pdf',
 'http://e-mjm.org/2012/v67n1/Polymorphism.pdf',
 'http://e-mjm.org/2020/v75n1/ingested-pins.pdf',
 'http://eprints.usm.my/35038/1/BULLETIN_23_PART_3.pdf'

In [19]:
usim = filtered_pdfs

In [21]:
# !rm -rf usim
!mkdir usim

In [22]:
max_worker = 1

usim = sorted(list(usim))
for i in tqdm(range(1, len(usim), max_worker)):
    urls = [(url, os.path.join('usim', f'{i}-{no}.pdf')) for no, url in enumerate(usim[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  1%|▎                                      | 78/9744 [01:27<8:13:36,  3.06s/it]

0 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2015/06/7-Tinjauan-akta-persaingan.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac25f40>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2015/06/7-Tinjauan-akta-persaingan.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac25f70>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2015/06/7-Tinjauan-akta-persaingan.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac25520>: Failed to establish a new connection: [Errno -2] Name or service not known'))


  1%|▎                                      | 79/9744 [01:29<7:29:39,  2.79s/it]

0 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2020/01/5-Orang-asli.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac25940>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2020/01/5-Orang-asli.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac254c0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='jurnalkanun.dbp.my', port=80): Max retries exceeded with url: /wordpress/wp-content/uploads/2020/01/5-Orang-asli.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feb2ac25460>: Failed to establish a new connection: [Errno -2] Name or service not known'))


  1%|▍                                      | 96/9744 [01:48<1:13:36,  2.18it/s]

...

  5%|█▊                                    | 462/9744 [21:59<4:13:27,  1.64s/it]

0 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/41739/5.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))
1 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/41739/5.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))
2 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/41739/5.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))


  5%|█▊                                    | 463/9744 [22:00<4:17:09,  1.66s/it]

0 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/74512/8.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))
1 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/74512/8.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))
2 HTTPSConnectionPool(host='myjurnal.mohe.gov.my', port=443): Max retries exceeded with url: /filebank/published_article/74512/8.pdf (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1131)')))


 34%|█████████████                          | 3270/9744 [40:52<36:17,  2.97it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
!du -hs usim

21G	usim
